#**Embeddings**

Hello everybody,
in this part we want to show you how to build an embedding matrix and how to use it your projects

In [ ]:

###### array modules ######
import pandas as pd
import numpy as np

###### basic modules ######
from collections import Counter
import tqdm, os, gc, subprocess, zipfile, gdown, pickle
#!pip install gdown
from pathlib import Path
from termcolor import colored

###### tensorflow modules ######
import tensorflow as tf
from tensorflow import keras

## Embedding Visualization Modules
from tensorboard.plugins import projector
import tensorboard

###### embedding creation modules ######
!pip install gensim==3.8.3
import gensim
from gensim.models import FastText
from gensim.models import Word2Vec

###### Text Processing Tools Modules ######
!pip install hazm
import spacy
from hazm import *
# in the future you can import step_1

     |████████████████████████████████| 24.2MB 137kB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
     |████████████████████████████████| 317kB 8.6MB/s 
     |████████████████████████████████| 1.4MB 9.8MB/s 
     |████████████████████████████████| 235kB 34.9MB/s 
  Created wheel for nltk: filename=nltk-3.3-cp36-none-any.whl size=1394469 sha256=a12a7701d146855b47843074dacca66ae35b4fbdbca4c6128f859142b7502a8f
  Stored in directory: /root/.cache/pip/wheels/d1/ab/40/3bceea46922767e42986aef7606a600538ca80de6062dc266c
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp36-cp36m-linux_x86_64.whl size=154368 sha256=7b3ab52bffdffcb96f8669f68c2c16eee0922ed1cdf360dcf984d6699db95e49
  Stored in directory: /root/.cache/pip/wheels/66/15/54/4510dce8bb958b1cdd2c47425cbd1e1eecc0480ac9bb1fb9ab
Successfully built nltk libwapiti
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninst

In [ ]:
## modules initialization ##
stemmer = Stemmer()
lemmatizer = Lemmatizer()
sent_tokenizer = SentenceTokenizer()
normalizer = Normalizer()
tokenizer = WordTokenizer()
stop_w_list = stopwords_list()

sample_string = '1 2 3 .به نام خدا. امروز کلاس پردازش زبان طبیعي داریم. من به شما خیر مقدم و تبریک میگم'
print('raw text            : ', sample_string)
print('normalizing text    : ', normalizer.normalize(sample_string))
print('sentence tokenizer  : ', sent_tokenizer.tokenize(sample_string))
print('tokenizing text     : ', tokenizer.tokenize(sample_string))

print('removing stop words : ', [word for word in sample_string.split(sep=' ') if word not in stop_w_list])
print('removing stop words2: ', [word for word in tokenizer.tokenize(sample_string) if word not in stop_w_list])

print('lemmatizer          : ', [lemmatizer.lemmatize(word) for word in sample_string.split(sep=' ')])
print('stemmer             : ', [stemmer.stem(word) for word in sample_string.split(sep=' ')])

print('norm lemmatizer     : ', [lemmatizer.lemmatize(word) for word in tokenizer.tokenize(normalizer.normalize(sample_string))])
print('norm stemmer        : ', [stemmer.stem(word) for word in tokenizer.tokenize(normalizer.normalize(sample_string))])




raw text            :  1 2 3 .به نام خدا. امروز کلاس پردازش زبان طبیعي داریم. من به شما خیر مقدم و تبریک میگم
normalizing text    :  ۱ ۲ ۳. به نام خدا. امروز کلاس پردازش زبان طبیعی داریم. من به شما خیر مقدم و تبریک میگم
sentence tokenizer  :  ['1 2 3 .به نام خدا.', 'امروز کلاس پردازش زبان طبیعي داریم.', 'من به شما خیر مقدم و تبریک میگم']
tokenizing text     :  ['1', '2', '3', '.', 'به', 'نام', 'خدا', '.', 'امروز', 'کلاس', 'پردازش', 'زبان', 'طبیعي', 'داریم', '.', 'من', 'به', 'شما', 'خیر', 'مقدم', 'و', 'تبریک', 'میگم']
removing stop words :  ['1', '2', '3', '.به', 'نام', 'خدا.', 'امروز', 'کلاس', 'پردازش', 'زبان', 'طبیعي', 'داریم.', 'خیر', 'مقدم', 'تبریک', 'میگم']
removing stop words2:  ['1', '2', '3', '.', 'نام', 'خدا', '.', 'امروز', 'کلاس', 'پردازش', 'زبان', 'طبیعي', '.', 'خیر', 'مقدم', 'تبریک', 'میگم']
lemmatizer          :  ['1', '2', '3', '.به', 'نام', 'خدا.', 'امروز', 'کلاس', 'پردازش', 'زبان', 'طبیعي', 'داریم.', 'من', 'به', 'شما', 'خیر', 'مقدم', 'و', 'تبریک', 'میگ']
stemmer         

## **Load and Preprocessing data**

The examples here are from [This link](https://colab.research.google.com/drive/16HDMoHsdP9GKRSOYCVZ-O9wC2LcNh-Kt?usp=sharing). For more Details you can check this notebook in future.
#### **Mounting Google Drive locally**
This is how to mount your Google Drive on your runtime using an authorization code, and how to write and read files there, supporting read, write, and moving files.

Note: When using the 'Mount Drive' button in the file browser, no authentication codes are necessary for notebooks that have only been edited by the current user.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def un_zip(path):
    if not os.path.exists(path):
      with zipfile.ZipFile(path+'.zip','r') as zip_ref:
          zip_ref.extractall(path)

def read_pickle(path):
  with open(path, 'rb') as in_file:
      data = pickle.load(in_file)
  return data

def load_csv(path):
  df = pd.read_csv(path)
  return df

def preprocessing_data(line, remove_stop_word=False,
                     lemmatize=False,
                     stem=False):
    line = normalizer.normalize(line)
    line = tokenizer.tokenize(line)
    if remove_stop_word:
        line = [word for word in line if word not in stop_w_list]
    if stem:
        line = [stemmer.stem(word) for word in line]
    if lemmatize:
        line = [lemmatizer.lemmatize(word) for word in line]
        line
    line = ' '.join(line)
    return line

## **Create Embedding**


#### **word embedding**



In [ ]:
def create_embeddings(sentences,embedding_path='defualt_path.gensimmodel', word_vec_type='fasttext',
                      **params):
    if word_vec_type == 'fasttext':
        model = FastText(sentences, **params)
    else:
        model = Word2Vec(sentences, **params)
    print('Saving the embedding models')
    model.wv.save_word2vec_format(embedding_path)
    return model

def fasttext(sentence_text,embedding_path,embedding_dim,MYSG,iteration):
   return create_embeddings(sentence_text,embedding_path=embedding_path + '.gensimmodel' ,
                            size=embedding_dim, word_vec_type='fasttext',
                            sg=MYSG, workers=5,
                            iter=iteration,min_count=20,
                            word_ngrams=1, min_n=2, max_n=8, bucket=2000000,)

def w2v(sentence_text,embedding_path,embedding_dim,MYSG,iteration):
        return create_embeddings(sentence_text,embedding_path=embedding_path + '.gensimmodel',
                                  size=embedding_dim, word_vec_type='w2v', window=10,
                                  sg=MYSG, workers=5,min_count=20,
                                  iter=iteration,)

#### **Test Embeddings**

In [ ]:
def test_most_similar_word_list(model,word_list,top_number=3):
  in_words_list = [word for word in word_list if word in model.wv.vocab.keys()]
  if len(in_words_list) == 0 :
    return
  word_not_in_vocab = [word for word in word_list if word not in in_words_list]
  print('these words are not in vocab',word_not_in_vocab)

  most_similar = [related_word[0] for related_word in model.wv.most_similar(positive=[word for word in in_words_list], topn=top_number)]
  print('most similar words to word_list : ',most_similar)
  most_similar_cosmul = [related_word[0] for related_word in model.wv.most_similar_cosmul(positive=[word for word in in_words_list], topn=top_number)]
  print('most similar cosmul words to word_list : ' , most_similar_cosmul)

def test_most_similar_word(model,word,top_number=3) :
  if word not in model.wv.vocab.keys():
    print('word is not in vocab')
    return
  print('most similar words : ',[related_word[0] for related_word in model.wv.similar_by_word(word=word, topn=top_number)])

def test_most_similar_analogy(model, in_word_1, in_word_2, out_word_1, out_word_2, top_number=3):
  if in_word_1 not in model.wv.vocab.keys():
    # print('first word is not in vocab')
    return -1
  if in_word_2 not in model.wv.vocab.keys():
    # print('second word is not in vocab')
    return -1
  if out_word_1 not in model.wv.vocab.keys():
    # print('third word is not in vocab')
    return -1

  in_1 = model.wv.vectors[model.wv.vocab[in_word_1].index]
  in_1_2 = in_1 / sum(in_1 ** 2) ** 0.5

  in_2 = model.wv.vectors[model.wv.vocab[in_word_2].index]
  in_2_2 = in_2 / sum(in_2 ** 2) ** 0.5

  out_1 = model.wv.vectors[model.wv.vocab[out_word_1].index]
  out_1_2 = out_1 / sum(out_1 ** 2) ** 0.5

  out_2_2 = in_2 + out_1 - in_1
  out_2_3 = in_2_2 + out_1_2 - in_1_2

  distance_by_vec = model.wv.similar_by_vector(out_2_2, topn=top_number)

  normalized_distance_by_vec = model.wv.similar_by_vector(out_2_3, topn=top_number)

  most_similar_distance = model.wv.most_similar(positive=[in_word_2, out_word_1], \
                                    negative=[in_word_1], topn=top_number)

  most_similar_cosmul_distance = model.wv.most_similar_cosmul(positive=[in_word_2, out_word_1],\
                                    negative=[in_word_1], topn=top_number)

  distance_by_vec = [related_word[0] for related_word in distance_by_vec]
  normalized_distance_by_vec = [related_word[0] for related_word in normalized_distance_by_vec]
  most_similar_distance = [related_word[0] for related_word in most_similar_distance]
  most_similar_cosmul_distance = [related_word[0] for related_word in most_similar_cosmul_distance]

  if out_word_2 in distance_by_vec:
    return 1
  else:
    return 0

## Visualizing Embedding

In [ ]:
def visualize_embedding(model, log_dir):
  Path(log_dir).mkdir(parents=True, exist_ok=True)
  with open(os.path.join(log_dir, 'metadata.tsv'), "w") as f:
      for subwords in model.wv.index2word:
          f.write("{}\n".format(subwords))

  # # Fill in the rest of the labels with "unknown"
  # for unknown in range(1, encoder.vocab_size - len(encoder.subwords)):
  #   f.write("unknown #{}\n".format(unknown))
  # Save the weights we want to analyse as a variable. Note that the first
  # value represents any unknown word, which is not in the metadata, so
  # we will remove that value.
  weights = tf.Variable(model.wv.vectors)
  # Create a checkpoint from embedding, the filename and key are
  # name of the tensor.
  checkpoint = tf.train.Checkpoint(embedding=weights)
  checkpoint.save(os.path.join(log_dir, "embedding.ckpt"))
  # Set up config
  config = projector.ProjectorConfig()
  embedding = config.embeddings.add()
  # The name of the tensor will be suffixed by `/.ATTRIBUTES/VARIABLE_VALUE`
  embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
  embedding.metadata_path = 'metadata.tsv'
  projector.visualize_embeddings(log_dir, config)
  input('see the result and if you pass press any key for continue')

## **Run & Results**


In [ ]:
save_path = "/content/drive/MyDrive/MasterPro/"
Path(save_path).mkdir(parents=True, exist_ok=True)

url = 'https://drive.google.com/u/0/uc?id=1HCPSk0NR16yMUA0UvKpDbMQ-TfSevd04&export=download'

output = save_path + 'Persian_Corpus.txt.zip'
gdown.download(url, output, quiet=False)

un_zip(save_path + 'Persian_Corpus.txt')
with open (save_path + 'Persian_Corpus.txt/Persian_Corpus.txt') as file:
  data = file.readlines()

Downloading...
From: https://drive.google.com/u/0/uc?id=1HCPSk0NR16yMUA0UvKpDbMQ-TfSevd04&export=download
To: /content/drive/MyDrive/MasterPro/Persian_Corpus.txt.zip
61.7MB [00:00, 99.1MB/s]


In [ ]:
data = data[:]

In [ ]:
cleaned_data = []
for doc in tqdm.tqdm(data):
  cleaned_data.append(preprocessing_data(doc, remove_stop_word=True,lemmatize=False,stem=False))
print('\n')
print('cleaned_line len is : ', len(cleaned_data))
print(cleaned_data[10])

  2%|▏         | 40182/2000000 [00:04<03:16, 9996.03it/s] 

In [ ]:
splited_line = [line.split(sep=' ') for line in cleaned_data if len(line.split(sep=' ')) > 3]
print(splited_line[0])
### for skipgram use 1 and for cbow use 0
MYSG = 0
#################### w2v #####################
save_name = 'w2v-50-10-12'
embedding_path = save_path + save_name
w2v_model = w2v(splited_line,embedding_path=embedding_path,embedding_dim = 50,MYSG=MYSG,iteration=12)
#########################################
# save_name = 'fasttext-100-1'
# embedding_path = save_path + save_name
# fastext_model = fasttext(splited_line,embedding_path=embedding_path,embedding_dim = 100,MYSG=MYSG,iteration=12)

['بسم', 'الله', 'الرحمن', 'الرحیم']
Saving the embedding models


In [ ]:
visualize_embedding(w2v_model, log_dir)

see the result and if you pass press any key for continuee


In [ ]:
log_dir = save_path + 'tensorboard/logs/' + 'w2v' + '/'
%tensorboard --logdir {log_dir}

In [ ]:
results = []
path = '/content/drive/MyDrive/'
embeddings = ['fasttext-100-0.gensimmodel', 'fasttext-100-1.gensimmodel', 'fasttext-50-0.gensimmodel', 'fasttext-50-1.gensimmodel', 'w2v-100-10-12.gensimmodel','w2v-100-5-12.gensimmodel', 'w2v-50-10-12.gensimmodel', 'w2v-50-5-12.gensimmodel']

for model_conf in embeddings:
    embedding_path = path + model_conf

    model = gensim.models.KeyedVectors.load_word2vec_format(embedding_path)

    p = [0, 0, 0, 0]
    for i, top_n in enumerate([1, 5, 10, 20]):
        with open('Test_file.txt') as test:
            corrects = 0
            counts = 0
            for line in test.readlines():
                in_word_1, in_word_2, out_word_1, out_word_2 = line[:-1].split(',')[1:]
                res = test_most_similar_analogy(model, in_word_1, in_word_2,
                                                out_word_1, out_word_2, top_number=top_n)
                if res != -1:
                    corrects += res
                    counts += 1

        p[i] = corrects / counts
    print(model_conf)
    results.append({'model_type': model_conf,
                    'Top-1': p[0],
                    'Top-5': p[1],
                    'Top-10': p[2],
                    'Top-20': p[3]})


In [ ]:
for i in range(8):
  print(results[i])